#### Batch Normalization

In [5]:
import keras.losses
from keras.layers import Flatten, Dense, BatchNormalization
from keras.models import Sequential

In [6]:
model = Sequential([
    Flatten(input_shape=[28, 28]),
    BatchNormalization(),
    Dense(300, activation="relu"),
    BatchNormalization(),
    Dense(100, activation="relu"),
    BatchNormalization(),
    Dense(10, activation="softmax")
])

2022-03-16 17:47:18.308025: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 17:47:18.372677: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 17:47:18.372771: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-16 17:47:18.374292: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)              4

In [8]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

#### Gradient Clipping

In [9]:
from keras.optimizers import gradient_descent_v2

optimizer = gradient_descent_v2.SGD(clipvalue=1.0)

In [10]:
import numpy as np
from keras.datasets import fashion_mnist

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6)  # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2  # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32)  # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))


(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [11]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [12]:
X_train_A.shape

(43986, 28, 28)

In [13]:
X_train_B.shape

(200, 28, 28)

In [14]:
model_A = Sequential()
model_A.add(Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(Dense(n_hidden, activation="selu"))
model_A.add(Dense(8, activation="softmax"))

In [15]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=gradient_descent_v2.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [16]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
 152/1375 [==>...........................] - ETA: 1s - loss: 1.2267 - accuracy: 0.6030

2022-03-16 17:47:20.971326: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1375/1375 [==============================] - 3s 1ms/step - loss: 0.5695 - accuracy: 0.8142 - val_loss: 0.3775 - val_accuracy: 0.8787
Epoch 2/20
1375/1375 [==============================] - 1s 1ms/step - loss: 0.3540 - accuracy: 0.8778 - val_loss: 0.3208 - val_accuracy: 0.8954
Epoch 3/20
1375/1375 [==============================] - 1s 1ms/step - loss: 0.3156 - accuracy: 0.8895 - val_loss: 0.3110 - val_accuracy: 0.8926
Epoch 4/20
1375/1375 [==============================] - 1s 1ms/step - loss: 0.2952 - accuracy: 0.8972 - val_loss: 0.2843 - val_accuracy: 0.9026
Epoch 5/20
1375/1375 [==============================] - 1s 1ms/step - loss: 0.2815 - accuracy: 0.9019 - val_loss: 0.2745 - val_accuracy: 0.9073
Epoch 6/20
1375/1375 [==============================] - 1s 1ms/step - loss: 0.2708 - accuracy: 0.9060 - val_loss: 0.2695 - val_accuracy: 0.9098
Epoch 7/20
1375/1375 [==============================] - 1s 1ms/step - loss: 0.2629 - accuracy: 0.9084 - val_loss: 0.2648 - val_accuracy: 0.9096
Epo

In [17]:
model_A.save("my_model_A.h5")

In [18]:
model_B = Sequential()
model_B.add(Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(Dense(n_hidden, activation="selu"))
model_B.add(Dense(1, activation="sigmoid"))

In [19]:
model_B.compile(loss="binary_crossentropy",
                optimizer=gradient_descent_v2.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [20]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 0s 15ms/step - loss: 0.7091 - accuracy: 0.5450 - val_loss: 0.5881 - val_accuracy: 0.6815
Epoch 2/20
7/7 [==============================] - 0s 10ms/step - loss: 0.5596 - accuracy: 0.7500 - val_loss: 0.4990 - val_accuracy: 0.7718
Epoch 3/20
7/7 [==============================] - 0s 8ms/step - loss: 0.4744 - accuracy: 0.8250 - val_loss: 0.4309 - val_accuracy: 0.8448
Epoch 4/20
7/7 [==============================] - 0s 7ms/step - loss: 0.4096 - accuracy: 0.8850 - val_loss: 0.3750 - val_accuracy: 0.8966
Epoch 5/20
7/7 [==============================] - 0s 8ms/step - loss: 0.3553 - accuracy: 0.9250 - val_loss: 0.3331 - val_accuracy: 0.9138
Epoch 6/20
7/7 [==============================] - 0s 7ms/step - loss: 0.3145 - accuracy: 0.9450 - val_loss: 0.2993 - val_accuracy: 0.9361
Epoch 7/20
7/7 [==============================] - 0s 8ms/step - loss: 0.2825 - accuracy: 0.9600 - val_loss: 0.2729 - val_accuracy: 0.9473
Epoch 8/20
7/7 [================

In [21]:
model_B.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_9 (Dense)             (None, 300)               235500    
                                                                 
 dense_10 (Dense)            (None, 100)               30100     
                                                                 
 dense_11 (Dense)            (None, 50)                5050      
                                                                 
 dense_12 (Dense)            (None, 50)                2550      
                                                                 
 dense_13 (Dense)            (None, 50)                2550      
                                                                 
 dense_14 (Dense)            (None, 1)                

In [22]:
from keras.models import load_model

model_A: Sequential = load_model("my_model_A.h5")
model_B_on_A = Sequential(model_A.layers[:-1])  # no output layer
model_B_on_A.add(Dense(1, activation="sigmoid"))

# Note that model_B_on_A and model_A share layers now, so when we train one, it will update both models. If we want to avoid that, we need to build model_B_on_A on top of a clone of model_A:

In [23]:
from keras.models import clone_model

model_A_clone = clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(Dense(1, activation="sigmoid"))

In [24]:
for layer in model_B_on_A.layers[:-1]:  # Freezing all hidden layers
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [25]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 15ms/step - loss: 0.1134 - accuracy: 0.9750 - val_loss: 0.1219 - val_accuracy: 0.9736
Epoch 2/4
7/7 [==============================] - 0s 10ms/step - loss: 0.0988 - accuracy: 0.9850 - val_loss: 0.1100 - val_accuracy: 0.9757
Epoch 3/4
7/7 [==============================] - 0s 10ms/step - loss: 0.0881 - accuracy: 0.9850 - val_loss: 0.1006 - val_accuracy: 0.9807
Epoch 4/4
7/7 [==============================] - 0s 9ms/step - loss: 0.0797 - accuracy: 0.9850 - val_loss: 0.0919 - val_accuracy: 0.9838


In [26]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=gradient_descent_v2.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 0s 17ms/step - loss: 0.0714 - accuracy: 0.9850 - val_loss: 0.0839 - val_accuracy: 0.9888
Epoch 2/16
7/7 [==============================] - 0s 9ms/step - loss: 0.0647 - accuracy: 0.9900 - val_loss: 0.0778 - val_accuracy: 0.9899
Epoch 3/16
7/7 [==============================] - 0s 9ms/step - loss: 0.0594 - accuracy: 0.9950 - val_loss: 0.0734 - val_accuracy: 0.9899
Epoch 4/16
7/7 [==============================] - 0s 8ms/step - loss: 0.0557 - accuracy: 0.9950 - val_loss: 0.0695 - val_accuracy: 0.9899
Epoch 5/16
7/7 [==============================] - 0s 7ms/step - loss: 0.0522 - accuracy: 0.9950 - val_loss: 0.0658 - val_accuracy: 0.9899
Epoch 6/16
7/7 [==============================] - 0s 7ms/step - loss: 0.0490 - accuracy: 0.9950 - val_loss: 0.0630 - val_accuracy: 0.9909
Epoch 7/16
7/7 [==============================] - 0s 7ms/step - loss: 0.0465 - accuracy: 0.9950 - val_loss: 0.0606 - val_accuracy: 0.9919
Epoch 8/16
7/7 [=================

In [27]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 654us/step - loss: 0.1327 - accuracy: 0.9835


[0.13267095386981964, 0.9835000038146973]

In [28]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 0.0390 - accuracy: 0.9985


[0.03900434821844101, 0.9984999895095825]

#### Momentum Optimization

In [29]:
_ = gradient_descent_v2.SGD(momentum=0.9)

#### Nesterov Accelerated Gradient

In [30]:
_ = gradient_descent_v2.SGD(momentum=0.9, nesterov=True)

#### AdaGrad

In [31]:
from keras.optimizers import adagrad_v2

_ = adagrad_v2.Adagrad()  # not recommended for nn, since it can stop to early

#### RMSProp

In [32]:
from keras.optimizers import rmsprop_v2

_ = rmsprop_v2.RMSprop(learning_rate=0.001, rho=0.9)

#### Adam and Nadam
nadam is Adam + Nesterov Trick

In [33]:
from keras.optimizers import adam_v2

_ = adam_v2.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

#### Learning Rate Schedulers

#### Power Scheduling

In [34]:
_ = gradient_descent_v2.SGD(learning_rate=0.01, decay=1e-4)  # decay adds power scheduling

#### Exponential scheduling

In [35]:
# with hardcoded values
def exponential_decay_fn(epoch: int) -> float:
    return 0.01 * 0.1 ** (epoch / 20)


# with input values
def exponential_decay(lr0: float, s: int):
    def exponential_decay_fn(epoch: int) -> float:
        return lr0 * 0.1 ** (epoch / s)

    return exponential_decay_fn


exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

In [36]:
model = Sequential([
    Flatten(input_shape=[28, 28]),
    Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 25

In [38]:
from keras.callbacks import LearningRateScheduler

lr_scheduler = LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train_scaled, y_train, epochs=n_epochs, validation_data=(X_valid_scaled, y_valid),
                    callbacks=[lr_scheduler])

Epoch 1/25
1719/1719 [==============================] - 3s 1ms/step - loss: 0.8394 - accuracy: 0.7605 - val_loss: 0.8355 - val_accuracy: 0.7960 - lr: 0.0100
Epoch 2/25
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6491 - accuracy: 0.8035 - val_loss: 0.6450 - val_accuracy: 0.8380 - lr: 0.0089
Epoch 3/25
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6314 - accuracy: 0.8072 - val_loss: 0.9756 - val_accuracy: 0.7168 - lr: 0.0079
Epoch 4/25
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5329 - accuracy: 0.8321 - val_loss: 0.6138 - val_accuracy: 0.8156 - lr: 0.0071
Epoch 5/25
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4874 - accuracy: 0.8453 - val_loss: 0.6354 - val_accuracy: 0.7814 - lr: 0.0063
Epoch 6/25
1719/1719 [==============================] - 3s 1ms/step - loss: 0.4329 - accuracy: 0.8615 - val_loss: 0.5133 - val_accuracy: 0.8442 - lr: 0.0056
Epoch 7/25
1719/1719 [==============================] - 2s

In [39]:
# decay will start at the beginning of epoch 0 instead of 1. Will rely upon existing lr (e.g. existing model)
def exponential_decay_fn(epoch: int, lr: float) -> float:
    return lr * 0.1 ** (epoch / 20)

#### Piecewise constant scheduling

In [40]:
def piecewise_constant_fn(epoch: int) -> float:
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

#### Performance scheduling

In [41]:
from keras.callbacks import ReduceLROnPlateau

_ = ReduceLROnPlateau(factor=0.5, patience=5)

In [47]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay

s = 20 * len(X_train) // 32  # number of steps in 20 epochs (batch size = 32)
learning_rate = ExponentialDecay(0.01, s, 0.1)
optimizer = gradient_descent_v2.SGD(learning_rate)